In [1]:
# for loading the csv  file
from google.colab import files
import io
############## general dependencies ####################################
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
import sklearn as skl
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
#distinct value counter
from collections import Counter

In [2]:
data = files.upload()

Saving diamonds.csv to diamonds.csv


In [3]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
# Load the data into a Pandas DataFrame
diamonds_df= pd.read_csv(io.StringIO(data['diamonds.csv'].decode('utf-8')))

# Display sample data
diamonds_df.head(10)
# Review the DataFrame

,carat,cut,color,clarity,depth,table,price,'x','y','z'
0,0.23,b'Ideal',b'E',b'SI2',61.5,55.0,326.0,3.95,3.98,2.43
1,0.21,b'Premium',b'E',b'SI1',59.8,61.0,326.0,3.89,3.84,2.31
2,0.23,b'Good',b'E',b'VS1',56.9,65.0,327.0,4.05,4.07,2.31
3,0.29,b'Premium',b'I',b'VS2',62.4,58.0,334.0,4.20,4.23,2.63
4,0.31,b'Good',b'J',b'SI2',63.3,58.0,335.0,4.34,4.35,2.75
5,0.24,b'Very Good',b'J',b'VVS2',62.8,57.0,336.0,3.94,3.96,2.48
6,0.24,b'Very Good',b'I',b'VVS1',62.3,57.0,336.0,3.95,3.98,2.47
7,0.26,b'Very Good',b'H',b'SI1',61.9,55.0,337.0,4.07,4.11,2.53
8,0.22,b'Fair',b'E',b'VS2',65.1,61.0,337.0,3.87,3.78,2.49
9,0.23,b'Very Good',b'H',b'VS1',59.4,61.0,338.0,4.00,4.05,2.39


In [4]:
#Checking datatypes
diamonds_df.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price      float64
'x'        float64
'y'        float64
'z'        float64
dtype: object

Cleaning the Data and removing columns without effect

In [5]:
#removing the  "b'" in front of cut,color and clarity as this might not be helpful
diamonds_df['color'] = diamonds_df['color'].str.replace("b'",'')
diamonds_df['cut'] =diamonds_df['cut'].str.replace("b'", '')
diamonds_df['clarity'] = diamonds_df['clarity'].str.replace("b'", '')
diamonds_df['color'] = diamonds_df['color'].str.replace("'", '')
diamonds_df['cut'] = diamonds_df['cut'].str.replace("'", '')
diamonds_df['clarity'] =diamonds_df['clarity'].str.replace("'", '')
diamonds_df

,carat,cut,color,clarity,depth,table,price,'x','y','z'
0,0.23,Ideal,E,SI2,61.5,55.0,326.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757.0,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757.0,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757.0,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757.0,6.15,6.12,3.74


In [6]:
#removing unwanted columns and creating a df with no duplicates
df_no_dup = diamonds_df.drop(columns = ['cut',"'x'", "'y'", "'z'"])
df_no_dup

#(df_no_dup = diamonds_df.drop(columns = ['depth', 'table', "'x'", "'y'", "'z'"]))

,carat,color,clarity,depth,table,price
0,0.23,E,SI2,61.5,55.0,326.0
1,0.21,E,SI1,59.8,61.0,326.0
2,0.23,E,VS1,56.9,65.0,327.0
3,0.29,I,VS2,62.4,58.0,334.0
4,0.31,J,SI2,63.3,58.0,335.0
...,...,...,...,...,...,...
53935,0.72,D,SI1,60.8,57.0,2757.0
53936,0.72,D,SI1,63.1,55.0,2757.0
53937,0.70,D,SI1,62.8,60.0,2757.0
53938,0.86,H,SI2,61.0,58.0,2757.0


In [7]:
# stage 1 changing clarity str no numerical values, first seeig how many values are present
df_no_dup['clarity'].value_counts()

SI1     13065
VS2     12258
SI2      9194
VS1      8171
VVS2     5066
VVS1     3655
IF       1790
I1        741
Name: clarity, dtype: int64

In [8]:
# Convert the numeric object values to string values 
df_no_dup['clarity'].replace(['IF','VVS1','VVS2','VS1','VS2','SI1','SI2','I1'],[1,2,3,4,5,6,7,8],inplace=True)
df_no_dup.head()

,carat,color,clarity,depth,table,price
0,0.23,E,7,61.5,55.0,326.0
1,0.21,E,6,59.8,61.0,326.0
2,0.23,E,4,56.9,65.0,327.0
3,0.29,I,5,62.4,58.0,334.0
4,0.31,J,7,63.3,58.0,335.0


In [9]:
# 2. Check the datatypes of your columns with the newly changed 'clarity' column and compare to diamonds_df. clarity's dtype is not int64 
df_no_dup.dtypes

carat      float64
color       object
clarity      int64
depth      float64
table      float64
price      float64
dtype: object

Changing the diamond colour from string to int so our model can process it better

In [10]:
#find out how many different values we have 
df_no_dup['color'].value_counts()

G    11292
E     9797
F     9542
H     8304
D     6775
I     5422
J     2808
Name: color, dtype: int64

In [11]:
# Convert the numeric object values to string values, the higher the number the more colourful is diamond is. lower number means a diamonds is almost colourless and therefore ideal
df_no_dup['color'].replace(['G','E','F','H','D','I','J'],[10,20,30,40,50,60,70],inplace=True)
df_no_dup.head()

,carat,color,clarity,depth,table,price
0,0.23,20,7,61.5,55.0,326.0
1,0.21,20,6,59.8,61.0,326.0
2,0.23,20,4,56.9,65.0,327.0
3,0.29,60,5,62.4,58.0,334.0
4,0.31,70,7,63.3,58.0,335.0


Now that we had cleaned up our data, we are going to begin setting up for our model's training.

In [12]:
#input data of regressor
#X= np.array(df_no_dup["price"]).reshape(-1,1)
#X.shape

In [13]:
#y=np.array(df_no_dup)
#y.shape

#### Step 2: Create the labels set (y) from the “price” column, and then create the features (X) DataFrame from the remaining columns.
#### NOTE
#### A value of 1 in the “clarity” column means that the diamond is high quality . A value of 8 means that the diamonds has flaws visible to the naked eye.
##### Split the data into training and testing datasets by using train_test_split

In [14]:
# Separate the data into labels and features

# Separate the y variable, the labels
y =df_no_dup['price']

# Separate the X variable, the features
X =df_no_dup.drop(columns=['price'])

In [15]:
# Review the y variable Series
y

0         326.0
1         326.0
2         327.0
3         334.0
4         335.0
          ...  
53935    2757.0
53936    2757.0
53937    2757.0
53938    2757.0
53939    2757.0
Name: price, Length: 53940, dtype: float64

In [16]:
X

,carat,color,clarity,depth,table
0,0.23,20,7,61.5,55.0
1,0.21,20,6,59.8,61.0
2,0.23,20,4,56.9,65.0
3,0.29,60,5,62.4,58.0
4,0.31,70,7,63.3,58.0
...,...,...,...,...,...
53935,0.72,50,6,60.8,57.0
53936,0.72,50,6,63.1,55.0
53937,0.70,50,6,62.8,60.0
53938,0.86,40,7,61.0,58.0


attempt 2

In [32]:
#the input regressor
X= np.array(df_no_dup["price"]).reshape(-1,1)
X

array([[ 326.],
       [ 326.],
       [ 327.],
       ...,
       [2757.],
       [2757.],
       [2757.]])

In [34]:
X.shape

(53940, 1)

In [35]:
#output data predictor
y=np.array(df_no_dup["carat"])
y

array([0.23, 0.21, 0.23, ..., 0.7 , 0.86, 0.75])

In [36]:
#what is the shape of y now
y.shape

(53940,)

In [37]:
print(type(y))

<class 'numpy.ndarray'>


In [38]:
print(type(X))

<class 'numpy.ndarray'>


In [39]:
#splitting dataset
X_train, X_test,y_train,y_test= train_test_split(X,y,test_size= 0.30)

In [43]:
#explore the train and test datasets
X_train.shape

(37758, 1)

In [41]:
X_test.shape

(16182, 1)

In [45]:
y_train.shape

(37758,)

In [44]:
y_test.shape

(16182,)

#modeling

In [47]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LinearRegression

In [48]:
#modelling instantiation
model= LinearRegression()
#fitting model
mymodel=model.fit(X_train,y_train)


In [56]:
#getting attributes for the model and printing them
print("the intercept is:",mymodel.intercept_)
print("the slope is:",mymodel.coef_)

y_r=print("the coeff of determindation is:",mymodel.score(X_train,y_train))


the intercept is: 0.3662736529104613
the slope is: [0.00010982]
the coeff of determindation is: 0.8500231627938499


In [57]:
#predicting using testing
y_pred=mymodel.predict(X_test)
y_pred

array([0.53715368, 2.1702879 , 0.51947265, ..., 0.83795084, 0.42195243,
       0.42019531])

TypeError: ignored

attempt2

Create a Logistic Regression Model with the Original Data
Step 1: Fit a logistic regression model by using the training data (X_train and y_train).

In [17]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test,y_train,y_test =train_test_split(X,y, random_state=1)

In [60]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile and train

In [72]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
diamonds_nn= len(X_train_scaled[0])
hidden_node_layer = 120
second_hidden_node_layer =80
third_hidden_layer= 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node_layer, input_dim=diamonds_nn, activation='relu'))
#additional layer
nn.add(tf.keras.layers.Dense(units=third_hidden_layer, input_dim=diamonds_nn, activation='sigmoid'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_hidden_node_layer, activation='sigmoid'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 120)               240       
                                                                 
 dense_17 (Dense)            (None, 30)                3630      
                                                                 
 dense_18 (Dense)            (None, 80)                2480      
                                                                 
 dense_19 (Dense)            (None, 1)                 81        
                                                                 
Total params: 6,431
Trainable params: 6,431
Non-trainable params: 0
_________________________________________________________________


In [73]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [75]:
# Train the model
trained_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1180/1180 [==============================] - 2s 2ms/step - loss: -26.8706 - accuracy: 0.0289
Epoch 2/100
1180/1180 [==============================] - 2s 2ms/step - loss: -31.7491 - accuracy: 0.0289
Epoch 3/100
1180/1180 [==============================] - 3s 3ms/step - loss: -36.6193 - accuracy: 0.0289
Epoch 4/100
1180/1180 [==============================] - 3s 2ms/step - loss: -41.5124 - accuracy: 0.0289
Epoch 5/100
1180/1180 [==============================] - 2s 2ms/step - loss: -46.4184 - accuracy: 0.0289
Epoch 6/100
1180/1180 [==============================] - 2s 2ms/step - loss: -51.3162 - accuracy: 0.0289
Epoch 7/100
1180/1180 [==============================] - 2s 2ms/step - loss: -56.2106 - accuracy: 0.0289
Epoch 8/100
1180/1180 [==============================] - 3s 3ms/step - loss: -61.0799 - accuracy: 0.0289
Epoch 9/100
1180/1180 [==============================] - 3s 2ms/step - loss: -65.9702 - accuracy: 0.0289
Epoch 10/100
1180/1180 [==============================]

In [70]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

506/506 - 1s - loss: -2.1535e+01 - accuracy: 0.0289 - 1s/epoch - 2ms/step
Loss: -21.53515625, Accuracy: 0.028859226033091545


In [23]:
# Export our model to HDF5 file
#nn.save('../saved_files/AlphabetSoupCharity.h5')

In [24]:
#exporting our DDF5 File
#files.download('/saved_files/AlphabetSoupCharity.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>